In [70]:
input_form = """

<table>
<tr>
<td style="border-style: none;">

<div style="border: solid 2px #666; width: 143px; height: 144px;">

<canvas width="140" height="140"></canvas>

</div></td>

<td style="border-style: none;">

<button onclick="set_value()">Predict</button>

<button onclick="clear_value()">Clear</button>

</td>
</tr>
<tr>
<td>
예측:<input type="text" id="result">
</td>
</tr>
</table>

"""



javascript = """

<script type="text/Javascript">

    var pixels = [];

    for (var i = 0; i < 28*28; i++) pixels[i] = 0

    var click = 0;



    var canvas = document.querySelector("canvas");

    canvas.addEventListener("mousemove", function(e){
    
         if (e.buttons == 1) {

            canvas.getContext("2d").fillStyle = "rgb(0,0,0)";

            canvas.getContext("2d").fillRect(e.offsetX, e.offsetY, 8, 8);

            x = Math.floor(e.offsetY * 0.2)

            y = Math.floor(e.offsetX * 0.2) + 1

            for (var dy = 0; dy < 2; dy++){

                for (var dx = 0; dx < 2; dx++){

                    if ((x + dx < 28) && (y + dy < 28)){

                        pixels[(y+dy)+(x+dx)*28] = 1

                    }

                }

            }
        }

    });

    

    function set_value(){

        var result = ""

        for (var i = 0; i < 28*28; i++) result += pixels[i] + ","

        var kernel = IPython.notebook.kernel;
        
        kernel.execute("image = [" + result + "]");
        kernel.execute("image = np.array(image).reshape(-1, 28, 28)");
        
        // Python 코드의 실행 결과를 받아오는 callback 함수를 지정
        kernel.execute("predict = str(np.argmax(model.predict(image)))");
        kernel.execute("IPython.notebook.kernel.execute('predict = ' + predict)");
        document.getElementById('result').value = predict;
    }

    

    function clear_value(){

        canvas.getContext("2d").fillStyle = "rgb(255,255,255)";

        canvas.getContext("2d").fillRect(0, 0, 140, 140);

        for (var i = 0; i < 28*28; i++) pixels[i] = 0

    }

</script>

"""

In [1]:
import tensorflow as tf

# 1. MNIST 데이터셋 임포트
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 2. 데이터 전처리
x_train, x_test = x_train/255.0, x_test/255.0

# 3. 모델 구성
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

# 4. 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 5. 모델 훈련
model.fit(x_train, y_train, epochs=5)

# 6. 정확도 평가
test_loss, test_acc = model.evaluate(x_test, y_test)
print('테스트 정확도:', test_acc)

11490434/11490434 [==============================] - 2s 0us/step
Epoch 1/5
1875/1875 [==============================] - 24s 12ms/step - loss: 0.1976 - accuracy: 0.9413
Epoch 2/5
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0794 - accuracy: 0.9758
Epoch 3/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0522 - accuracy: 0.9836
Epoch 4/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0373 - accuracy: 0.9885
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.0656 - accuracy: 0.9793
테스트 정확도: 0.9793000221252441


In [71]:
from IPython.display import HTML

HTML(input_form + javascript)

#### 사용법
- 위에서 숫자를 그리고 Predict클릭
- 다음 셀의 코드 실행

In [58]:
import numpy as np

image = np.array(image).reshape(-1, 28, 28)
ret = np.argmax(model.predict(image))
ret

1/1 [==============================] - 0s 55ms/step


3